In [ ]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import connect
import json
import pandas as pd
import pandas_redshift as pr

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'ENTER json filename'
VIEW_ID = 'ENTER GA ID'
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
service = build('analyticsreporting', 'v4', credentials=credentials)

In [ ]:
DIMS = ['ga:date', 'ga:medium']
METRICS = ['ga:newUsers',]

response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2022-01-01', 'endDate': 'today'}],
          'dimensions': [{'name': name} for name in DIMS],
          'metrics': [{'expression': exp} for exp in METRICS],
          'pageSize': 999999999
        }]
      }
  ).execute()

In [ ]:
data_dic = {f"{i}": [] for i in DIMS + METRICS}
for report in response.get('reports', []):
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        for i, key in enumerate(DIMS):
            data_dic[key].append(row.get('dimensions', [])[i]) 
        dateRangeValues = row.get('metrics', [])
        for values in dateRangeValues:
            all_values = values.get('values', [])
            for i, key in enumerate(METRICS):
                data_dic[key].append(all_values[i])
        
df = pd.DataFrame(data=data_dic)
df.columns = [col.split(':')[-1] for col in df.columns]

In [ ]:
df['date'] = df['date'].astype('datetime64[ns]')
df['newUsers'] = df['newUsers'].astype('int64')
df.rename(columns = {'newUsers':'new_users'}, inplace = True)
df
df.to_csv('ga_new_users.csv',sep='\t')

In [ ]:
unique_mediums = pd.DataFrame(df.medium.unique())  
unique_mediums 
df.to_csv('unique_mediums.csv',sep='\t')